In [2]:
import pandas as pd
from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS
from path import Path
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# LexRank Test File

Following along with the [LexRank Tutorial](https://github.com/crabcamp/lexrank).

In [3]:
docs = []
doc_dir_tng = Path('../StarTrekNextGenScriptData/scripts_tng')
doc_dir_dsn = Path('../StarTrekNextGenScriptData/scripts_ds9')
doc_dirs = [doc_dir_dsn,doc_dir_tng]

# open sentences too
complete = pd.read_csv('../StarTrekNextGenScriptData/complete_data.csv')

In [4]:
# using the ENTIRE script, which may not have useful data (lots of empty lines)
for documents_dir in doc_dirs:
    for file_path in documents_dir.files('*.txt'):
        with file_path.open(mode='rt', encoding='latin-1') as fp:
            docs.append(fp.readlines())

In [5]:
short_docs = docs[:50]

In [6]:
lxr = LexRank(docs, stopwords=STOPWORDS['en'])


In [7]:
sentences = list(complete[complete.file=='508.txt']['quote'])

In [8]:
summary = lxr.get_summary(sentences, summary_size=5, threshold=.1)
summary

['   So in the meantime, he\'s going to    keep having...       "visions."       I knew we were headed for trouble    the minute he allowed the Bajorans    to call him "Emissary."',
 "   Chief, I know you're worried. But    the Prophets are leading the    Emissary on this path for a    reason...",
 '   The Prophets chose well when they    made you their Emissary.       So how does it all fit together?',
 "   The Captain's not going to die.    He's the Emissary. The Prophets    will take care of him.",
 '   May the Prophets reveal their    wisdom to you, Emissary.']